# Optimizers

Optimizers are used to fit the model weights in neural networks, using the loss function.
In this notebook, we will go through every popular optimizers, from elaboration to codes.

In [ ]:
## Data Setup

import numpy as np

def data_generate():
    # Generate synthetic data (for example, y = x**2 + 1)
    np.random.seed(0)
    
    X = 2 * np.random.rand(100, 1)
    X_b = np.column_stack((np.ones(100), X))
    Y = 2 * X + 1 + np.random.randn(100, 1)
    

    # Initialize parameters (weights)
    w = np.random.randn(2, 1)
    
    return X_b, Y, w

### 0. Gradient

$$ g(w) = \frac{\delta L(x,w)}{\delta w} $$
Where:

$L(x,w)$: Loss

If use Loss function for MSE:

$$L(x,w) = \frac{1}{m}\sum^{m}_{i=1}(\hat{y}_i - y_i)^2
$$

As we are using x to predict the line $y=2x+1$, we will calculate two weights: $w_0$ for y-intercept, $w_1$ for slope.

which means that our $\hat{y}$ can be converted into $w_0 + w_1 x$. Plugging the number, we can calculate the formula:

$$ 
\begin{aligned}

g(w) &= \frac{\delta L(x,w)}{\delta w} \\
&= \frac{\delta}{\delta w}\frac{1}{m}\sum^{m}_{i=1}(\hat{y}_i - y_i)^2 \\
&= \frac{\delta}{\delta w}\frac{1}{m}\sum^{m}_{i=1}(w_0 + w_1x - y_i)^2 \\
&= \frac{2}{m}\sum^{m}_{i=1}(w_0 + w_1x_i - y_i)\cdot (w_0 + w_1x_i - y_i)'\\
g(w_0) &= \frac{2}{m}\sum^{m}_{i=1}(w_0 + w_1x_i - y_i)\cdot 1\\
g(w_1) &= \frac{2}{m}\sum^{m}_{i=1}(w_0 + w_1x_i - y_i)\cdot x_i
\end{aligned}
$$

$$
X = \begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\vdots \\
x_m
\end{bmatrix}, \; X_b = \begin{bmatrix}
1 \qquad x_1 \\
1 \qquad x_2 \\
1 \qquad x_3 \\
\vdots \\
1 \qquad x_m
\end{bmatrix}, Y = \begin{bmatrix}
y_1 \\
y_2 \\
y_3 \\
\vdots \\
y_m
\end{bmatrix}\; W = \begin{bmatrix}
w_0 \\
w_1 \\
\end{bmatrix} \\

X_b^T \cdot W = \begin{bmatrix}
w_0 + w_1x_1 = \hat{y_1}\\
w_0 + w_1x_2 = \hat{y_2}\\
\vdots \\
w_0 + w_1x_m = \hat{y_m}
\end{bmatrix},

\frac{2}{m} \cdot X_b^T \cdot (X_b^T \cdot W - Y) = \begin{bmatrix}
g(w_0) \\
g(w_1)
\end{bmatrix}
$$


In [ ]:
def gradient(X, Y, w):
    m = len(X)
    return 2 / m * X.T.dot((X.T - Y).dot(w))

### 1. Gradient Descent


$$ w_{t+1} = w_t - \eta g(w_t) $$

Where:

$\eta$: learning rate

$w_t$: t-th weight 

$g(w_t)$: Gradient of the loss function at weight $w_t$

In gradient descent, the goal is to update the weights in a way that minimizes the loss. The direction of the update is determined by the gradient, which points in the direction of the steepest increase in the loss. The magnitude of the update is controlled by the learning rate, denoted as $\eta$. Typically, the learning rate decreases over time in a process known as learning rate scheduling, where it starts at a higher value and gradually decreases to a lower value as training progresses.

In each iteration of gradient descent, all input data points are used to compute the loss, and the weights are updated accordingly to minimize this loss.



In [ ]:
def MSELoss(X, Y, w):
    m = len(X)
    return (1/m) * np.sum((X.dot(w)-Y) ** 2)

def gradient_descent(X, Y, w, learning_rate = 0.1, max_iterations = 1000, epsilon = 1e-6):
    loss_history = []
    
    for t in range(1, max_iterations+1):
        grad = gradient(X, Y, w)
        w_new = w - learning_rate * grad 
        
        if np.linalg.norm(w_new - w) < epsilon: # np.linalg.norm(): 벡터의 norm을 구하는 함수
            print(f"converged at iteration {t}.")
            break
        w = w_new
        loss = MSELoss(X, Y, w)
        loss_history.append(loss)
        
    return w, loss_history

### 2. Stochastic Gradient Descent (SGD)

$$ w_{t+1} = w_t - \eta g(w_t) $$

Same function with gradient descent, but the methodology of calculating the loss has been changed.

The problem of the gradient descent is that calculating all x inputs in every iteration is computationally expensive, so picking a random set of the number (small batch, from one to many) and tune the data can make it more efficient.



In [ ]:
def stochastic_gradient_descent(X, Y, w, learning_rate = 0.01, max_iterations = 1000, batch_size = 1, epsilon = 1e-6):
    loss_history = []
    m = len(X)
    
    for t in range(1, max_iterations+1):
        shuffled_indices = np.random.permutation(m) # generates from 0 to m-1 randomly shuffled number array.
        X_shuffled = X[shuffled_indices]
        Y_shuffled = Y[shuffled_indices]
        
        for j in range(0, m, batch_size):
            X_batch = X_shuffled[j:j+batch_size]
            Y_batch = Y_shuffled[j:j+batch_size]
            
            grad = gradient(X_batch, Y_batch, w)
            w_new = w - learning_rate * grad 
            
            if np.linalg.norm(w_new - w) < epsilon: # np.linalg.norm(): 벡터의 norm을 구하는 함수
                print(f"converged at iteration {t}.")
                return w, loss_history
            w = w_new
        loss = MSELoss(X, Y, w)
        loss_history.append(loss)
        
    return w, loss_history

### 3. SGD with momentum

$$ w_{t+1} = w_{t} + v_{t+1} \\
v_{t+1} = \rho v_{t} - \eta g(w_t) \\
w_{t+1} = w_{t} + \rho v_{t+1}
$$ 

$v$: velocity

$\rho$: momentum

One of the problem in the SGD is that it is hard to say "this weight is not the local minimum of loss, but it is the global minimum of loss". Because using SGD, the weight is likely to get trapped into the local minimum solution of the loss. 

One solution for the problem is to add the velocity; the momentum to let weight learn from the previous move, and let it flow from the local minimum.


In [ ]:
def SGD_with_momentum(X, Y, w, learning_rate = 0.01, momentum = 0.9, max_iterations = 1000, batch_size = 1, epsilon = 1e-6):
    loss_history = []
    m = len(X)
    v = 0
    
    for t in range(1, max_iterations+1):
        shuffled_indices = np.random.permutation(m) # generates from 0 to m-1 randomly shuffled number array.
        X_shuffled = X[shuffled_indices]
        Y_shuffled = Y[shuffled_indices]
        
        for j in range(0, m, batch_size):
            X_batch = X_shuffled[j:j+batch_size]
            Y_batch = Y_shuffled[j:j+batch_size]
            
            grad = gradient(X_batch, Y_batch, w)
            v_new = momentum * v - learning_rate * grad
            w_new = w + v_new
            
            if np.linalg.norm(w_new - w) < epsilon: # np.linalg.norm(): 벡터의 norm을 구하는 함수
                print(f"converged at iteration {t}.")
                return w, loss_history
            v = v_new
            w = w_new
        loss = MSELoss(X, Y, w)
        loss_history.append(loss)
    return w, loss_history

### 3-1. SGD with Nesterov momentum

$$ v_{t+1} = \rho v_{t} - \eta g(w_t + \rho v_t) \\
w_{t+1} = w_t + v_{t+1} $$

SGD with momentum calculates the slope in the current location. It might cause the overshooting. So Nesterov momentum tries to solve the issue by calculating the slope in the next location, by calculating the next step's location. So it shoots the weight in more accurate direction.

In [ ]:
def SGD_with_Nesterov_momentum(X, Y, w, learning_rate = 0.01, momentum = 0.9, max_iterations = 1000, batch_size = 1, epsilon = 1e-6):
    loss_history = []
    m = len(X)
    v = 0
    
    for t in range(1, max_iterations+1):
        shuffled_indices = np.random.permutation(m) # generates from 0 to m-1 randomly shuffled number array.
        X_shuffled = X[shuffled_indices]
        Y_shuffled = Y[shuffled_indices]
        
        for j in range(0, m, batch_size):
            X_batch = X_shuffled[j:j+batch_size]
            Y_batch = Y_shuffled[j:j+batch_size]
            
            grad = gradient(X_batch, Y_batch, w + momentum * v)
            v_new = momentum * v - learning_rate * grad
            w_new = w + v_new
            
            if np.linalg.norm(w_new - w) < epsilon: # np.linalg.norm(): 벡터의 norm을 구하는 함수
                print(f"converged at iteration {t}.")
                return w, loss_history
            v = v_new
            w = w_new
        loss = MSELoss(X, Y, w)
        loss_history.append(loss)
    return w, loss_history

### 4. AdaGrad

$$ \omega_{i, t+1} = \omega_{i, t} - \frac{\eta}{\epsilon + \sqrt{\sum_{j=1}^{t}g(\omega_{i,j})^2}}g(\omega_{i ,t})$$

$\epsilon$: numerical stablizer to prevent zero division

Schedules are most of the times expoenetially reduced and w gets closer to the target minimum (or local minimum, hopefully not). This is the problem with the decaying (reduced learning rate throughout the iteration) learning methods. It is decayed because it is assumed to get closer to the target, but it may not be, and even the learning rate may fizzled out even the hypothetical target value is far away.

So AdaGrad applies learning rate separately in every parameters. More updated parameters get lower learning rate, and less updated parameters remain higher learning rate.

In [ ]:
def ada_grad(X, Y, w, learning_rate = 0.01, max_iterations = 1000, epsilon = 1e-6):
    loss_history = []
    m = len(X)
    G = 0 # G represents the total sum of the squared gradients
    for t in range(1, max_iterations+1):
        grad = gradient(X, Y, w)
        G += grad ** 2
        adjusted_learning_rate = w - learning_rate / (epsilon + np.sqrt(G))
        w_new = w - adjusted_learning_rate * grad
        if np.linalg.norm(w_new - w) < epsilon:
            print(f"converged at iteration {t}.")
            break
        w = w_new
        loss = MSELoss(X, Y, w)
        loss_history.append(loss)
    return w, loss_history

### 5. RMSprop

$$ v_{t+1} = \beta v_t + (1-\beta)g(w_t)^2 \\
w_{t+1} = w_t - \frac{\eta}{\epsilon + \sqrt{v_{t+1}}}g(w_t) $$

$\beta$: discount parameter

AdaGrad may decay faster than the target. Root mean squared propagation attempts to solve this problem by allowing the effective learning rate to both decrease and increase.

The discount parameter determines how much of the previous v term is remembered.
Small gradeient is scaled up, and the big gradient is encountered, it is scaled down.

In [ ]:
def RMSprop(X, Y, w, learning_rate = 0.01, beta = 0.9, max_iterations = 1000, epsilon = 1e-6):
    loss_history = []
    m = len(X)
    v = 0
    for t in range(1, max_iterations+1):
        grad = gradient(X, Y, w)
        v_new = beta*v + (1-beta) * grad**2
        w_new = w - learning_rate / (epsilon + np.sqrt(v_new)) * grad
        if np.linalg.norm(w_new - w) < epsilon:
            print(f"converged at iteration {t}.")
            break
        v = v_new
        w = w_new
        loss = MSELoss(X, Y, w)
        loss_history.append(loss)
    return w, loss_history

### 6. Adaptive Moment Estimation (Adam)

$$ m_{t+1} = \beta_1 m_t + (1 - \beta_1) g(w_t) \\
v_{t+1} = \beta_2 v_t + (1-\beta_2) g(w_t)^2 \\
w_{t+1} = w_t - \frac{\eta}{\epsilon + \sqrt{\hat{v}_{t+1}}}\hat{m}_{t+1} \\
\hat{m}_t = \frac{m_t}{1-\beta_1^t} \\
\hat{v}_t = \frac{v_t}{1-\beta_2^t} \\
$$

It utilizes the both benefits from Momentum, and RMS Prop.
- Momentum: it adds up the inertia using the slopes.
- RMSProp: it optimizes the learning rate using the slopes.

Adam uses the two estimates from the two methods to tune the learning rate.

In [ ]:
def adam(X, Y, w, learning_rate = 0.01, beta1 = 0.9, beta2 = 0.99, max_iterations = 1000, epsilon = 1e-6):
    loss_history = []
    m = 0
    v = 0
    for t in range(1, max_iterations+1):
        grad = gradient(X, Y, w)
        m_new = beta1*m + (1-beta1) * grad
        v_new = beta2*v + (1-beta2) * grad**2
        m_hat_new = m_new / (1 - beta1)**t
        v_hat_new = v_new / (1 - beta2)**t
        
        w_new = w - learning_rate / (epsilon + np.sqrt(v_hat_new)) * m_hat_new
        
        if np.linalg.norm(w_new - w) < epsilon:
            print(f"converged at iteration {t}.")
            break
        
        v = v_new
        w = w_new
        loss = MSELoss(X, Y, w)
        loss_history.append(loss)
        
    return w, loss_history